# Import

In [11]:
import numpy as np
import pickle
import json
from sklearn.metrics import balanced_accuracy_score

# Common variables

In [37]:
DSs = ["cifar10", "iNaturalist"]

exp_types = ["benchmark", "solution1", "solution2"]

original_classes = {"cifar10": {
                        "plane": 0,
                        "car": 1,
                        "bird": 2,
                        "cat": 3,
                        "deer": 4,
                        "dog": 5,
                        "frog": 6,
                        "horse": 7,
                        "ship": 8,
                        "truck": 9
                    },
                    "iNaturalist": {
                        "agaricomycetes": 33,
                        "arachnida": 2,
                        "bryopsida": 37,
                        "insecta": 6,
                        "malacostraca": 7,
                        "actinopterygii": 9,
                        "amphibia": 10,
                        "aves": 12,
                        "mammalia": 14,
                        "reptilia": 15,
                        "magnoliopsida": 48,
                        "pinopsida": 49,
                        "polypodiopsida": 50,
                        "liliopsida": 46,
                        "bivalvia": 23,
                        "gastropoda": 25,
                        "lecanoromycetes": 29
                    }}

DS_labels = {"cifar10":    {"known": (0, 1, 2, 3, 4),
                            "unknown": (5, 6, 7, 8, 9)},
             "iNaturalist": {"known": (33, 2, 37, 6, 7, 9, 10, 12),
                            "unknown": (14, 15, 48, 49, 50, 46, 23, 25, 29)}}


# Read accuracy

## Automatic

In [3]:
for DS in DSs:
    print(f"Dataset: {DS}")
    print("=====================================================")
    for exp_type in exp_types:
        acc_overall, acc_known, acc_unknown = 0, 0, 0
        with open(f"../results/{DS}/{DS}_{exp_type}_batches_results.pkl", "rb") as f:
            results = pickle.load(f)
            for result in results:
                acc_overall += result["accuracy"]["overall"]
                acc_known += result["accuracy"]["known"]
                acc_unknown += result["accuracy"]["unknown"]
            
            print(f"{exp_type}:")
            print(f"Overall:\t{acc_overall/len(results)}")
            print(f"Known:\t\t{acc_known/len(results)}")
            print(f"Unknown:\t{acc_unknown/len(results)}")
            print("---")

Dataset: cifar10
benchmark:
Overall:	0.5242171306939817
Known:		0.4725799065525269
Unknown:	0.5758543548354363
---
solution1:
Overall:	0.5933383380384477
Known:		0.6385764740147798
Unknown:	0.5481002020621156
---
solution2:
Overall:	0.709023490604416
Known:		0.777684122766981
Unknown:	0.6403628584418514
---
Dataset: iNaturalist
benchmark:
Overall:	0.6747888673099379
Known:		0.729775087172662
Unknown:	0.6259122274319608
---
solution1:
Overall:	0.7297091815165114
Known:		0.823011028861259
Unknown:	0.6467742060989581
---
solution2:
Overall:	0.8760626128509751
Known:		0.9647122216515055
Unknown:	0.7972629605838368
---


## Refined

In [38]:
for DS in DSs:
    print(f"Dataset: {DS}")
    print("=====================================================")
    known_labels = DS_labels[DS]["known"]
    unknown_labels = DS_labels[DS]["unknown"]
    for exp_type in exp_types:
        acc_overall, acc_known, acc_unknown = 0, 0, 0
        with open(f"../results/{DS}/{DS}_{exp_type}_batches_results.pkl", "rb") as f:
            results = pickle.load(f)
        with open(f"../results/{DS}/merged_classes/{DS}_{exp_type}_pred_classes.json") as json_file:
            merged_classes = json.load(json_file)
        for i in range(len(results)):
            batch_result = results[i]
            pred_classes = batch_result["predicted_classes"]
            gt_labels = batch_result["gt_labels"]
            merged_dict = merged_classes[f"batch_{i}"]
            refined_pred_labels = list()
            for c in pred_classes:
                try:
                    refined_pred_labels.append(merged_dict[c])
                except:
                    refined_pred_labels.append(original_classes[DS][c])
            refined_pred_labels = np.array(refined_pred_labels)
            # Overall
            acc_overall += balanced_accuracy_score(gt_labels, refined_pred_labels)
            # # known
            desiered_items = [l in known_labels for l in gt_labels]
            acc_known += balanced_accuracy_score(gt_labels[desiered_items], refined_pred_labels[desiered_items])
            # # unknown
            desiered_items = [not(l in known_labels) for l in gt_labels]
            acc_unknown += balanced_accuracy_score(gt_labels[desiered_items], refined_pred_labels[desiered_items])
                    
        print(f"{exp_type}:")
        print(f"Overall:\t{acc_overall/len(results)}")
        print(f"Known:\t\t{acc_known/len(results)}")
        print(f"Unknown:\t{acc_unknown/len(results)}")
        print("---")
        

Dataset: cifar10
benchmark:
Overall:	0.805757550645807
Known:		0.7970933667812906
Unknown:	0.8144217345103234
---
solution1:
Overall:	0.759608186552752
Known:		0.7746170963007533
Unknown:	0.7445992768047507
---
solution2:
Overall:	0.8316906417863881
Known:		0.8535681250808658
Unknown:	0.8098131584919105
---
Dataset: iNaturalist
benchmark:
Overall:	0.8366852302691843
Known:		0.918904910795335
Unknown:	0.7636010698014944
---


c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_tr

solution1:
Overall:	0.8608983569948183
Known:		0.9569401029423693
Unknown:	0.775527916152551
---
solution2:
Overall:	0.8911198478661934
Known:		0.9647122216515055
Unknown:	0.8257044045014714
---


c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\alshouha\.python_venvs\transformers\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_tr

# Read results

In [18]:
with open(f"../results/original_classes.json", "w") as outfile: 
    json.dump(original_calsses, outfile, indent=4)

In [19]:
DS = "iNaturalist" # "cifar10" or "iNaturalist"

In [5]:
folder_path = f"../results/{DS}"

In [6]:
for exp_type in exp_types[2:]:
    print(f"Processing {exp_type}")
    full_results = dict()
    with open(f"{folder_path}/{DS}_{exp_type}_batches_results.pkl", "rb") as f:
        results = pickle.load(f)

        for i in range(len(results)):
            full_results[f"batch_{i}"] = results[i]["novel_classes_labels"]
        with open(f"{folder_path}/{DS}_{exp_type}_pred_classes.json", "w") as outfile:
            json.dump(full_results, outfile, indent=4)


Processing solution2


# Read merged classes

In [53]:
DS = "iNaturalist" # "cifar10" or "iNaturalist"
folder_path = f"../results/{DS}"

In [79]:
exp_type = "solution2"

with open(f"{folder_path}/merged_classes/{DS}_{exp_type}_pred_classes.json") as json_file:
    merged_classes = json.load(json_file)


In [82]:
Vs = merged_classes["batch_2"].values()
print(len(Vs))
print(len(set(Vs)))

13
12


In [83]:
for v in set(Vs):
    keys_with_value_v = [key for key, value in merged_classes["batch_2"].items() if value == v]
    list_of_objects = ", ".join(keys_with_value_v)
    print(f"{v}\t:\t{list_of_objects}")

12	:	holothuroidea
13	:	leotiomycetes
46	:	liliopsida
14	:	mammalia
48	:	magnoliopsida
49	:	pinopsida
50	:	polypodiopsida, equisetopsida
15	:	reptilia
22	:	none
23	:	bivalvia
25	:	gastropoda
29	:	none of the provided options fit.  the image shows a lichen, which is not an organism from a single biological class.  lichen are a symbiotic relationship between a fungus and an alga.  the fungal component would be classified


In [24]:
merged_classes["batch_2"]

{'dog': 5,
 'llama': 33,
 'horse': 7,
 'rabbit': 174,
 'boat': 8,
 'truck': 9,
 'hummingbird': 2,
 'mushroom': 56,
 'frog': 6,
 'flowers': 46,
 'dumper': 9,
 'pug': 5,
 'bicycle': 175,
 'antelope': 4,
 'puppy': 5,
 'yacht': 8,
 'airplane': 0,
 'convertible': 1,
 'ship': 8,
 'crab.': 176,
 'kitten': 3,
 'emu': 2,
 'seaplane': 0,
 'aircraft': 0,
 'taxi': 1,
 'microphone': 177,
 'fighterjet': 0,
 'goat': 35,
 'watch': 178,
 'waterfall': 62,
 'cabbage': 179,
 'gearshift.': 8,
 'rocks': 66,
 'carrots': 180,
 'oatmeal': 181,
 'shoes': 182,
 'parrot': 2,
 'pekingese': 5,
 'train': 26,
 'clamp': 183,
 'vehicle': 1,
 'horsebackrider': 7,
 'cassowary': 2,
 'bulldozer': 9,
 'refrigerator': 184,
 'pony': 7,
 'statue': 25,
 'skull': 27,
 'clouds': 185,
 'tugboat': 8,
 'cave': 186,
 'firetruck': 9,
 'bouquet': 187,
 'donkey': 188,
 'eye': 82,
 'ostrich': 2,
 'cow': 189,
 'fur': 190,
 'rooster': 2,
 'camaro': 1,
 'lioness': 50,
 'egg': 191,
 'fountain': 192,
 'camera': 193,
 'feces': 194,
 'house': 1

In [19]:
for exp_type in exp_types:
    with open(f"{folder_path}/merged_classes/{DS}_{exp_type}_pred_classes.json") as json_file:
        merged_classes = json.load(json_file)  
    print("========================================")
    print(exp_type)
    print("-----------")
    for batch_k, batch_v in merged_classes.items():
        print()
        print(batch_k)
        Vs = batch_v.values()
        for v in list(set(Vs)):
            keys_with_value_v = [key for key, value in merged_classes["batch_0"].items() if value == v]
            list_of_objects = ", ".join(keys_with_value_v)
            print(f"{v}\t:\t{list_of_objects}")

benchmark
-----------

batch_0
2	:	owl, ostrich
4	:	moose
5	:	dog
6	:	frog
7	:	horse
8	:	boat
9	:	truck
11	:	lizard
16	:	donkey
18	:	goat
19	:	building
21	:	submarine
22	:	fish
23	:	rock

batch_1
1	:	
2	:	owl, ostrich
4	:	moose
5	:	dog
6	:	frog
7	:	horse
8	:	boat
9	:	truck
12	:	
14	:	
19	:	building
23	:	rock
24	:	
27	:	
28	:	
30	:	
31	:	
32	:	
33	:	

batch_2
2	:	owl, ostrich
4	:	moose
5	:	dog
6	:	frog
7	:	horse
8	:	boat
9	:	truck
16	:	donkey
17	:	
18	:	goat
20	:	
21	:	submarine
22	:	fish
25	:	
26	:	
28	:	
solution1
-----------

batch_0
2	:	owl, ostrich
4	:	moose
5	:	dog
6	:	frog
7	:	horse
8	:	boat
9	:	truck
11	:	lizard
16	:	donkey
18	:	goat
19	:	building
21	:	submarine
22	:	fish
23	:	rock

batch_1
1	:	
2	:	owl, ostrich
4	:	moose
5	:	dog
6	:	frog
7	:	horse
8	:	boat
9	:	truck
12	:	
14	:	
19	:	building
23	:	rock
24	:	
27	:	
28	:	
30	:	
31	:	
32	:	
33	:	

batch_2
2	:	owl, ostrich
4	:	moose
5	:	dog
6	:	frog
7	:	horse
8	:	boat
9	:	truck
16	:	donkey
17	:	
18	:	goat
20	:	
21	:	submarine
22	:	f